# Test a sentence input
Requires these files in the same folder as this notebook:
- a bigram_phraser
- a trigram_phraser
- a word2vec model (3 files: model, syn1neg, and vectors)
- a list of stopwords (to ignore as potential euphemisms)

Required packages:
- gensim
- transformers

In [3]:
from gensim.models.phrases import Phraser, Phrases
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
import urllib.request
import numpy as np
from tqdm import tqdm
import re
import csv

# for discarding overly similar paraphrases
from difflib import SequenceMatcher
def get_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [7]:
class Euph_Detection:
    def __init__(self, bigram_phraser, trigram_phraser, w2v_model, stopwords_text, sentiment, offensive):
        self.bigram_phraser = Phraser.load(bigram_phraser)
        self.trigram_phraser = Phraser.load(trigram_phraser)
        self.base_model = Word2Vec.load(w2v_model)
        self.model = Word2Vec.load(w2v_model)
        # may move the below topic list to another file in the future
        self.topic_list = ['politics', 'death', 'kill', 'crime',
               'drugs', 'alcohol', 'fat', 'old', 'poor', 'cheap',
               'sex', 'sexual',
               'employment', 'job', 'disability', 'disabled', 
               'accident', 'pregnant', 'poop', 'sickness', 'race', 'racial', 'vomit'
              ]
        self.stopwords = self.read_stopwords(stopwords_text)
        # load the sentiment models and pack them together for conciseness
        self.sentiment_pack = [x for x in self.load_roberta(sentiment)]
        self.offensive_pack = [x for x in self.load_roberta(offensive)]
    
    def preprocess(self, s):
        s = s.strip()
        s = re.sub(r'(##\d*\W)|<\w>|,|;|:|--|\(|\)|#|%|\\|\/|\.|\*|\+|@', '', s)
        s = re.sub(r'\s\s+', ' ', s)
        s = s.lower()
        return s

    def get_phrases(self, s):
        bigrammed_phrases = self.bigram_phraser[s.split()]
        trigrammed_phrases = self.trigram_phraser[bigrammed_phrases]

    def sum_similarity(self, phrase, topic_list):
        score = 0
        for topic in topic_list:
            try:
                similarity = self.model.wv.similarity(phrase, topic)
                # "reward" the phrases with a high similarity to a particular category, but maybe not others
                if (similarity > 0.50):
                    return 1.51
                if (similarity > 0):
                    score += similarity
            except:
                score += 0
        return score
    
    def read_stopwords(self, text):
        stopwords = []
        with open(text,'rb') as f:
            content = f.read()
            content = content.split(b'\r\n')
            for line in content:
                stopwords.append(line.decode('utf-8'))
        return stopwords

    def topically_filter_phrases(self, phrases, topic_list, stopwords, THRESHOLD, show_stats=False):
        quality_phrases = []
        filtered = []

        for phrase in phrases:
            if (phrase in stopwords):
                continue
            similarity = self.sum_similarity(phrase, topic_list)

            if (show_stats == True):
                print("{} has a relevance score of {}".format(phrase, similarity)) #table?

            if (similarity > THRESHOLD and phrase not in quality_phrases):
                quality_phrases.append(phrase)
            else:
                filtered.append(phrase)

        # if (show_stats == True):
        #     print("\nRELEVANT PHRASES: {}".format(quality_phrases))
        #     print("IRRELEVANT PHRASES: {}".format(filtered))
        return quality_phrases

    def load_roberta(self, task):
        # Tasks:
        # emoji, emotion, hate, irony, offensive, sentiment
        # stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary
        # task='sentiment' or 'offensive'
        
        MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

        tokenizer = AutoTokenizer.from_pretrained(MODEL)

        # download label mapping
        labels=[]
        mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
        with urllib.request.urlopen(mapping_link) as f:
            html = f.read().decode('utf-8').split("\n")
            csvreader = csv.reader(html, delimiter='\t')
        labels = [row[1] for row in csvreader if len(row) > 1]

        # pretrained
        model = AutoModelForSequenceClassification.from_pretrained(MODEL)
        model.save_pretrained(MODEL)
        tokenizer.save_pretrained(MODEL)

        return labels, model, tokenizer
        
    '''
    functions for getting the sentiment 
    '''
    def get_sentiment(self, s, pack):
        labels, model, tokenizer = pack[0], pack[1], pack[2]
        encoded_input = tokenizer(s, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        return scores
    
    '''
    needs functions load_roberta_sentiment(), load_roberta_offensive(), get_sentiment() and get_offensive()
    '''
    def get_top_euph_candidates(self, text, phrases, num_paraphrases, wv_model, sentiment_pack, offensive_pack, show_stats=False):
        orig_scores = list(self.get_sentiment(text, sentiment_pack))
        orig_scores = orig_scores + list(self.get_sentiment(text, offensive_pack))
        
        if show_stats == True: print('SENTIMENT OF ORIGINAL SENTENCE: {}'.format(orig_scores))
        phrase_scores = []

        for candidate in tqdm(phrases):
            paraphrases = []
            if show_stats == True: print('\n'+candidate)
            paraphrases = wv_model.wv.most_similar(candidate, topn = num_paraphrases) # can swap out

            # print(paraphrases)

            # various sentiment statistics
            sentiment_shift = [0, 0, 0, 0, 0]
            max_inc = [0, 0, 0, 0, 0]
            max_inc_para = ["", "", "", "", ""]
            tot_neg_inc = 0
            tot_neu_inc = 0
            tot_pos_inc = 0
            tot_off_inc = 0
            tot_noff_inc = 0

            # variables to compute the length ratio feature
            length_ratio = 0
            tot_para_length = 0
            num_para = 0

            for p in paraphrases:
                p_string = re.sub(r'_', ' ', p[0]) # the underscores are removed for sentiment computation - experiment?
                candidate_string = re.sub(r'_', ' ', candidate)

                '''FILTERING PARAPHRASES'''
                # 1. ignore this paraphrase if it's a superstring of the candidate
                # e.g., "horrible_man" is not a good paraphrase of "man"
                if (candidate_string in p_string):
                    continue

                # 2. ignore this paraphrase if it's too similar to the candidate
                # e.g., "horrible_man" is not a good paraphrase of "men"
                if (get_similarity(candidate_string, p_string) > 0.5):
                    continue

                # 3. ignore this paraphrase if it contains profanity, because it results in high sentiment shifts
                #    but should never be a substitute for a euphemism
                # TODO: include more profanity
                # if ('fuck' in p_string):
                #     continue

                # 4. ignore this paraphrase if it occurs less than 5 times in the model
                #    this helps to eliminate the impact of gibberish
                if (wv_model.wv.get_vecattr(p[0], 'count') < 5):
                    continue
                '''END PARAPHRASE FILTERING'''

                # replacement
                pattern = re.compile(r'\b'+candidate_string+r'\b', re.IGNORECASE)
                new_sentence = pattern.sub(p_string, text)
                # at this point, we could check the integrity of the paraphrase

                # get the sentiment/offensive scores for this paraphrase
                scores = list(self.get_sentiment(new_sentence, sentiment_pack))
                scores = scores + list(self.get_sentiment(new_sentence, offensive_pack))
                
                # update the quality phrase's sentiment statistics with the sentiment shifts from this paraphrase
                shifts = [0, 0, 0, 0, 0]
                for i in range(0, len(scores)):
                    shifts[i] = scores[i] - orig_scores[i]
                    sentiment_shift[i] += shifts[i]
                    if (shifts[i] > max_inc[i]):
                        max_inc[i] = shifts[i]
                        max_inc_para[i] = p_string

                # update the relevant scores for detection
                if (shifts[0] > 0):
                    tot_neg_inc += shifts[0]
                if (shifts[1] > 0):
                    tot_neu_inc += shifts[1]
                if (shifts[2] > 0):
                    tot_pos_inc += shifts[2]
                if (shifts[3] > 0):
                    tot_noff_inc += shifts[3]
                if (shifts[4] > 0):
                    tot_off_inc += shifts[4]

                # update counts for length ratio
                num_para += 1
                tot_para_length += len(p_string)

            # compute length ratio feature
            if (num_para != 0):
                avg_para_length = tot_para_length / num_para
                length_ratio = len(candidate_string) / avg_para_length
            # print(length_ratio)
            # break

            for val in sentiment_shift:
                val /= num_paraphrases
            if (show_stats == True):
                print("AVERAGE SENTIMENT SHIFTS: {}".format(sentiment_shift))
                print("MAX INCREASE FROM A PHRASE: {}".format(max_inc))
                print("PHRASES THAT CAUSED EACH ^: {}".format(max_inc_para))
                print("TOTAL NEGATIVE INCREASE: {}".format(tot_neg_inc))
                print("TOTAL NEUTRAL INCREASE: {}".format(tot_neu_inc))
                print("TOTAL NEUTRAL INCREASE: {}".format(tot_noff_inc))
                print("TOTAL OFFENSIVE INCREASE: {}".format(tot_off_inc))

            # compute the score using the weights from linear regression on the 5 sentiment scores and length ratio feature
            phrase_scores.append((candidate_string, 0.02183689*tot_neg_inc + 0.01949368*tot_neu_inc + 0.09130243*tot_noff_inc + 0.12983809 *tot_off_inc + 0.15030182*length_ratio))
            
        phrase_scores = list(sorted(phrase_scores, key=lambda x: x[1], reverse=True))
        return phrase_scores
    
    def detect_euphs(self, s, topic_threshold, num_paraphrases, show_stats=False):
        s = self.preprocess(s)
        print(s)

        bigrammed_phrases = self.bigram_phraser[s.split()]
        trigrammed_phrases = self.trigram_phraser[bigrammed_phrases]
        print("\nDETECTED PHRASES: {}".format(trigrammed_phrases))

        data = []
        data.append(trigrammed_phrases)
        # train model on input data
        self.model.train(data, total_examples=len(data), epochs=10)

        quality_phrases = self.topically_filter_phrases(trigrammed_phrases, self.topic_list, self.stopwords, topic_threshold, show_stats)
        print("\nRELEVANT PHRASES: {}".format(quality_phrases))

        candidate_list = self.get_top_euph_candidates(s, quality_phrases, num_paraphrases, 
                                            self.model, self.sentiment_pack, self.offensive_pack, 
                                            show_stats)
        
        self.model = self.base_model # reset
        
        return candidate_list

In [8]:
import time

start = time.time()

euph_detector = Euph_Detection('data/bigram_phraser_7', 'data/trigram_phraser_7', 
                               'data/wv_model_7', 'data/stopwords.txt', 
                               'sentiment', 'offensive')
print("Setup took {} seconds".format(time.time() - start))

Setup took 69.74314403533936 seconds


#### Input your sentence below

In [38]:
s = "we don't know how bad things will get morocco said but it's critical to limit the spread of the virus by honoring the calls for social distancing and by not rushing to the hospital if we feel a bit under the weather"
candidate_ranking = euph_detector.detect_euphs(s, topic_threshold=1.45, num_paraphrases=25, show_stats=False)

print("\nEUPH CANDIDATE RANKING: {}".format(candidate_ranking))

we don't know how bad things will get morocco said but it's critical to limit the spread of the virus by honoring the calls for social distancing and by not rushing to the hospital if we feel a bit under the weather

DETECTED PHRASES: ["we_don't", 'know_how', 'bad_things', 'will_get', 'morocco', 'said', 'but', "it's", 'critical', 'to', 'limit_the_spread', 'of', 'the', 'virus', 'by', 'honoring', 'the', 'calls', 'for', 'social_distancing', 'and', 'by', 'not', 'rushing', 'to', 'the', 'hospital', 'if_we', 'feel_a_bit', 'under_the_weather']

RELEVANT PHRASES: ['know_how', 'bad_things', 'critical', 'limit_the_spread', 'virus', 'social_distancing', 'rushing', 'hospital', 'if_we', 'feel_a_bit', 'under_the_weather']


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:24<00:00,  2.26s/it]


EUPH CANDIDATE RANKING: [('under the weather', 0.31994708311037745), ('limit the spread', 0.2076849393645232), ('social distancing', 0.18775501163216696), ('virus', 0.1715375309333847), ('feel a bit', 0.15883262481811816), ('rushing', 0.14860603544614343), ('know how', 0.1400280180507521), ('hospital', 0.11456183642643182), ('critical', 0.11427736258451968), ('if we', 0.10781739292150148), ('bad things', 0.0)]


## Analyzing the Process
After running Euph_Detection on a sentence, you can further look at the intermediate outputs for a specific candidate phrase:

#### Topic Relevance

In [44]:
test_phrase = 'virus'
similar_topics = []

topic_list = euph_detector.topic_list
model = euph_detector.model

score = 0
for topic in topic_list:
    similarity = model.wv.similarity(test_phrase, topic)
    if (similarity > 0.25):
        similar_topics.append(topic)
    if (similarity > 0):
        score += similarity
    print('{}: {}'.format(topic, similarity))

print('SIMILAR TOPICS: {}'.format(similar_topics))
print('TOTAL SCORE: {}'.format(score))

politics: -0.12178656458854675
death: 0.3376462459564209
kill: 0.37465864419937134
crime: 0.2576283812522888
drugs: 0.4533570110797882
alcohol: 0.315814733505249
fat: 0.2738639712333679
old: 0.13116423785686493
poor: 0.03396809101104736
cheap: 0.036798909306526184
sex: 0.18742014467716217
sexual: 0.13066315650939941
employment: -0.01607104018330574
job: 0.09143663942813873
disability: 0.1916699856519699
disabled: 0.31440800428390503
accident: 0.35817182064056396
pregnant: 0.2856043577194214
poop: 0.268268346786499
sickness: 0.4311104714870453
race: 0.05317165330052376
racial: 0.04802703857421875
vomit: 0.2797028422355652
SIMILAR TOPICS: ['death', 'kill', 'crime', 'drugs', 'alcohol', 'fat', 'disabled', 'accident', 'pregnant', 'poop', 'sickness', 'vomit']
TOTAL SCORE: 4.854554686695337


#### Sentiment

In [46]:
text = s
q = 'under_the_weather'

sentiment_pack = euph_detector.sentiment_pack
offensive_pack = euph_detector.offensive_pack
model = euph_detector.model

orig_scores = list(euph_detector.get_sentiment(s, sentiment_pack))
orig_scores = orig_scores + list(euph_detector.get_sentiment(text, offensive_pack))
print('SENTIMENT OF ORIGINAL SENTENCE: {}'.format(orig_scores))
phrase_scores = []
num_paraphrases=25
paraphrases = []
print('\n'+q)
paraphrases = model.wv.most_similar(q, topn = num_paraphrases) # can swap out

# various sentiment statistics
sentiment_shift = [0, 0, 0, 0, 0] # [neg, neu, pos, off, n-off]
max_inc = [0, 0, 0, 0, 0]
max_inc_para = ["", "", "", "", ""]
tot_neg_inc = 0
tot_neu_inc = 0
tot_pos_inc = 0
tot_noff_inc = 0
tot_off_inc = 0

length_ratio = 0
tot_para_length = 0
num_para = 0

for p in paraphrases:
    p_string = re.sub(r'_', ' ', p[0]) # the underscores are removed for sentiment computation - experiment?
    q_string = re.sub(r'_', ' ', q)

    '''FILTERING PARAPHRASES'''
    # filtering out paraphrases if they're a superstring
    if (q_string in p_string):
#                 print("Paraphrase is superstring, skipping!")
#                 print()
        continue

    # filtering out paraphrases if they're too similar
    if (get_similarity(q_string, p_string) > 0.5):
        continue

    if ('fuck' in p_string):
        continue

    if (model.wv.get_vecattr(p[0], 'count') < 5):
        continue

    '''END PARAPHRASE FILTERING'''

    # replacement
    pattern = re.compile(r'\b'+q_string+r'\b', re.IGNORECASE)
    new_sentence = pattern.sub(p_string, text)
    # at this point, we could check the integrity of the paraphrase

    # get the sentiment/offensive scores for this paraphrase
    # scores = list(self.get_sentiment(new_sentence, sentiment_labels, sentiment_model, sentiment_tokenizer))
    # scores = scores + list(self.get_offensive(new_sentence, offensive_labels, offensive_model, offensive_tokenizer))
    scores = list(euph_detector.get_sentiment(new_sentence, sentiment_pack))
    scores = scores + list(euph_detector.get_sentiment(new_sentence, offensive_pack))

    # update the quality phrase's sentiment statistics with the sentiment shifts from this paraphrase
    shifts = [0, 0, 0, 0, 0]
    for i in range(0, len(scores)):
        shifts[i] = scores[i] - orig_scores[i]
        sentiment_shift[i] += shifts[i]
        if (shifts[i] > max_inc[i]):
            max_inc[i] = shifts[i]
            max_inc_para[i] = p_string

    # update the relevant scores for detection
    if (shifts[0] > 0):
        tot_neg_inc += shifts[0]
    if (shifts[1] > 0):
        tot_neu_inc += shifts[1]
    if (shifts[2] > 0):
        tot_pos_inc += shifts[2]
    if (shifts[3] > 0):
        tot_noff_inc += shifts[3]
    if (shifts[4] > 0):
        tot_off_inc += shifts[4]
    
    print(p_string)
    print(shifts)
    # update counts for length ratio
    num_para += 1
    tot_para_length += len(p_string)

# compute length ratio feature
if (num_para != 0):
    avg_para_length = tot_para_length / num_para
    length_ratio = len(q_string) / avg_para_length
#         print(length_ratio)
#         break

for val in sentiment_shift:
    val /= num_paraphrases

print("AVERAGE SENTIMENT SHIFTS: {}".format(sentiment_shift))
print("MAX INCREASE FROM A PHRASE: {}".format(max_inc))
print("PHRASES THAT CAUSED EACH ^: {}".format(max_inc_para))
print("TOTAL NEGATIVE INCREASE: {}".format(tot_neg_inc))
print("TOTAL NEUTRAL INCREASE: {}".format(tot_neu_inc))
print("TOTAL NEUTRAL INCREASE: {}".format(tot_noff_inc))
print("TOTAL OFFENSIVE INCREASE: {}".format(tot_off_inc))
print("LENGTH RATIO: {}".format(length_ratio))

# phrase_scores.append((q_string, tot_neg_inc + tot_neu_inc + 2*(tot_noff_inc + tot_off_inc)))
# phrase_scores.append((q_string, 0.02337676*tot_neg_inc + 0.02267515*tot_neu_inc + 0.09802046*tot_noff_inc + 0.14442855*tot_off_inc))
# 0.01125929*tot_pos_inc
phrase_scores.append((q_string, 0.02183689*tot_neg_inc + 0.01949368*tot_neu_inc + 0.09130243*tot_noff_inc + 0.12983809 *tot_off_inc + 0.15030182*length_ratio))

# phrase_scores.append((q_string, max_inc[0] + 2*max_inc[4]))
# phrase_scores.append((q_string, tot_neg_inc + tot_neu_inc + 2*(tot_off_inc)))
phrase_scores = list(sorted(phrase_scores, key=lambda x: x[1], reverse=True))
print(phrase_scores)

SENTIMENT OF ORIGINAL SENTENCE: [0.57720906, 0.38477147, 0.038019482, 0.82404214, 0.17595792]

under_the_weather
out of sorts
[0.024811089, -0.019412309, -0.0053988285, 0.0049026012, -0.004902616]
very hungry
[0.021269977, -0.019729972, -0.0015399233, -0.021305442, 0.021305427]
bummed out
[0.08563852, -0.07549262, -0.010145988, -0.05189395, 0.051893875]
hungover
[0.02169627, -0.022473186, 0.00077697635, -0.027323782, 0.027323678]
really busy
[-0.02429837, 0.017496675, 0.006801784, 0.017013133, -0.017013296]
antsy
[0.00614655, -0.006093055, -5.3409487e-05, 0.0010698438, -0.0010699332]
frazzled
[0.061351597, -0.053379625, -0.00797216, -0.03724599, 0.03724593]
nauseated
[0.10907304, -0.097510934, -0.01156217, -0.045651138, 0.04565108]
homesick
[0.015963435, -0.015984803, 2.1286309e-05, -0.0056694746, 0.0056694746]
ashamed of myself
[0.102668166, -0.08913267, -0.013535535, -0.056426883, 0.056426764]
lightheaded
[0.046756208, -0.04097554, -0.0057807453, -0.029704332, 0.029704273]
really hun